In [1]:
# Imports
import os
import requests
import pandas as pd
import datetime
from twilio.rest import Client

In [2]:
# GLOBAL API Setup
ALPHA_API_KEY = os.environ.get('ALPHA_API_KEY')
NEWS_API_KEY = os.environ.get('NEWS_API_KEY')
TWILIO_ACCOUNT_SID = os.environ.get('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.environ.get('TWILIO_AUTH_TOKEN')
FROM_WPP = os.environ.get('FROM_WPP')
TO_WPP = os.environ.get('TO_WPP')
ALPHA_URL = 'https://www.alphavantage.co/query?'
NEWS_URL = 'https://newsapi.org/v2/everything'


In [3]:

# Stock variation calculation
def get_stock_variation(stock_symbol) -> bool:
    """Fetch stock variation percentage and check if it exceeds 5%."""
    alpha_params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': stock_symbol.upper(),
        'apikey': ALPHA_API_KEY
    }
    
    response = requests.get(ALPHA_URL, params=alpha_params)
    response.raise_for_status()
    raw_data = response.json()
    data = raw_data["Time Series (Daily)"]
    
    # Get sorted available dates from API
    dates = sorted(data.keys(), reverse=True)
    print(dates)
    
    try:
        yesterday_date = dates[0]
        two_days_ago_date = dates[1]
    except IndexError:
        print("Not enough data to calculate stock variation")
        return False
    
    # Fetch close prices
    yesterday_close = float(data[yesterday_date]["4. close"])
    two_days_close = float(data[two_days_ago_date]["4. close"])
    
    percentage_difference = (yesterday_close - two_days_close) / two_days_close * 100
    print(f'Stock price variation:{percentage_difference}')
    
    if abs(percentage_difference) >= 5:
        print("The variation is more than 5%")
        return True
    else:
        print("The variation is within 5%")
        return False

In [4]:
# Fetch news articles
def get_news(query) -> list:
    """Retrieve top 3 news articles based on the query."""
    
    yesterday_date = datetime.datetime.now() - datetime.timedelta(days=1)
    news_params = {
        'apiKey': os.environ.get('NEWS_API_KEY'),
        'q': query,
        'from': yesterday_date,
        'language': 'en',
        'sortBy':'relevancy'
    }
    
    response = requests.get(NEWS_URL, news_params)
    response.raise_for_status()
    data = response.json() # o: python dict
    # print(data)
    # print(type(data))

    return data['articles'][0:3]

In [5]:
# Send wpp message
def set_wpp_message(message):
    """Send a WhatsApp message using Twilio."""
    client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
    msn = client.messages.create( 
        body=message,
        from_=FROM_WPP,
        to=TO_WPP,
        )
    return msn.status

In [6]:
# TEST
stock_price_variation = get_stock_variation("AAPL")
print(stock_price_variation)
news = get_news("apple")
print(news)
message= "testing MESSAGE. From: test."
msn = set_wpp_message(message)

['2025-01-24', '2025-01-23', '2025-01-22', '2025-01-21', '2025-01-17', '2025-01-16', '2025-01-15', '2025-01-14', '2025-01-13', '2025-01-10', '2025-01-08', '2025-01-07', '2025-01-06', '2025-01-03', '2025-01-02', '2024-12-31', '2024-12-30', '2024-12-27', '2024-12-26', '2024-12-24', '2024-12-23', '2024-12-20', '2024-12-19', '2024-12-18', '2024-12-17', '2024-12-16', '2024-12-13', '2024-12-12', '2024-12-11', '2024-12-10', '2024-12-09', '2024-12-06', '2024-12-05', '2024-12-04', '2024-12-03', '2024-12-02', '2024-11-29', '2024-11-27', '2024-11-26', '2024-11-25', '2024-11-22', '2024-11-21', '2024-11-20', '2024-11-19', '2024-11-18', '2024-11-15', '2024-11-14', '2024-11-13', '2024-11-12', '2024-11-11', '2024-11-08', '2024-11-07', '2024-11-06', '2024-11-05', '2024-11-04', '2024-11-01', '2024-10-31', '2024-10-30', '2024-10-29', '2024-10-28', '2024-10-25', '2024-10-24', '2024-10-23', '2024-10-22', '2024-10-21', '2024-10-18', '2024-10-17', '2024-10-16', '2024-10-15', '2024-10-14', '2024-10-11', '2024

In [13]:
def main():
    stock_price_variation = get_stock_variation("EC")
    topic= "ecopetrol"
    if stock_price_variation:
        print("Stock variation detected. Fetching news articles...")
        
        news = get_news(topic)
        for article in news:
            message = f"Headline: {article['title']}\nBrief: {article['description']}\nLink: {article['url']}"
            set_wpp_message(message)
            print(f"Message sent: {message}")
    else:
        news = get_news(topic)
        print(news)
        message = f"Headline: {article['title']}\nBrief: {article['description']}\nLink: {article['url']}"
        set_wpp_message(message)
        print(f"Message sent: {message}")
        print("No stock variation detected")
        

In [14]:
main()

['2025-01-24', '2025-01-23', '2025-01-22', '2025-01-21', '2025-01-17', '2025-01-16', '2025-01-15', '2025-01-14', '2025-01-13', '2025-01-10', '2025-01-08', '2025-01-07', '2025-01-06', '2025-01-03', '2025-01-02', '2024-12-31', '2024-12-30', '2024-12-27', '2024-12-26', '2024-12-24', '2024-12-23', '2024-12-20', '2024-12-19', '2024-12-18', '2024-12-17', '2024-12-16', '2024-12-13', '2024-12-12', '2024-12-11', '2024-12-10', '2024-12-09', '2024-12-06', '2024-12-05', '2024-12-04', '2024-12-03', '2024-12-02', '2024-11-29', '2024-11-27', '2024-11-26', '2024-11-25', '2024-11-22', '2024-11-21', '2024-11-20', '2024-11-19', '2024-11-18', '2024-11-15', '2024-11-14', '2024-11-13', '2024-11-12', '2024-11-11', '2024-11-08', '2024-11-07', '2024-11-06', '2024-11-05', '2024-11-04', '2024-11-01', '2024-10-31', '2024-10-30', '2024-10-29', '2024-10-28', '2024-10-25', '2024-10-24', '2024-10-23', '2024-10-22', '2024-10-21', '2024-10-18', '2024-10-17', '2024-10-16', '2024-10-15', '2024-10-14', '2024-10-11', '2024

UnboundLocalError: cannot access local variable 'article' where it is not associated with a value